In [333]:
import numpy as np 
import pandas as pd
from sklearn import datasets
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Class of Neural Network with Affine Layer 

In [328]:
class NeuralNetwork:
    def __init__(self):
        self.temp=0
        self.weights=[]
        self.bias=[]
        self.outputs=0
        self.derivative_table=0
        self.model_loss=100
        self.local_derivatives=[]
    def initialize_weights_and_bais(self,X,layers,neurons):
        l=X.shape[1]
        for i,j in enumerate(neurons):
            self.weights.append(np.array([np.random.random(l) for k in range(j)]))
            self.bias.append(np.round(np.random.random(j),6))
            l=self.weights[i].shape[0]
    def ReLU(self,INPUT):
        INPUT[INPUT<0]=0
        derr=np.zeros_like(INPUT)
        derr[INPUT>0]=1
        return INPUT,derr
    def forward(self,X):
        a=X
        for i in range(len(self.weights)):
            dw=a
            a=np.dot(a,self.weights[i].T)+self.bias[i]
            dx=self.weights[i]
            db=np.ones(self.bias[i].shape)
            if i!=len(self.weights)-1:
                relu,relu_derr=self.ReLU(a)
            else :
                relu=np.array([0])
                relu_derr=np.array([0])
            self.local_derivatives.append([relu,dw,dx,db,relu_derr])
            if i!=len(self.weights)-1:
                a=relu
#             print('O : ',a.shape,' DW: ' ,dw.shape,' DX: ' ,dx.shape,' DB: ' ,db.shape,' relu: ',relu.shape,' derr: ',relu_derr.shape)
        return a
    
    def loss(self,y,outputs):
        self.derivative_table=outputs-outputs[np.arange(y.shape[0]),y].reshape(-1,1)
        self.derivative_table[self.derivative_table<=0]=0
        self.model_loss=np.sum(self.derivative_table)
        self.derivative_table[self.derivative_table>0]=1
        self.derivative_table[np.arange(y.shape[0]),y]=np.sum(self.derivative_table,axis=1)*-1
        return self.derivative_table
        
    def backward(self,LR,output):
        holding_weights=[]
        holding_bias=[]
        final_output=output
        for i in reversed(self.local_derivatives):
            if i[-1].shape[0]!=1:
                final_ouput=np.multiply(final_output,i[-1])
            
#             print('Ouput : ',final_output.shape)
#             print('DW : ',i[1].shape)
#             print('DX : ',i[2].shape)
#             print('DB : ',i[3].shape)
#             print('weights : ',np.dot(final_output.T,i[1]).shape)  # updated weights
#             print('bias : ',np.sum(final_output,axis=0).shape)  # updated bias
#             print('x : ',np.dot(final_output,i[2]).shape)
            
            holding_weights.append(np.dot(final_output.T,i[1]))
            holding_bias.append(np.sum(final_output,axis=0))
            final_output=np.dot(final_output,i[2])
#             print('\n')
            
        c=len(self.weights)-1
        for i in range(len(holding_weights)):
            self.weights[c]=self.weights[c]-(LR*holding_weights[i])
            self.bias[c]=self.bias[c]-(LR*holding_bias[i])
            c=c-1
            
    def train(self,X,y,layers,neurons,LR=0.0001,iterations=50000,loss_time=1000):

        self.initialize_weights_and_bais(X,layers,neurons)
        for i in range(iterations):
            OUT=self.forward(X)
            DERR=self.loss(y,OUT)
            self.backward(LR,DERR)

            self.local_derivatives=[]            # empty local derivatives to calculate them again
            if i%loss_time==0:
                print(self.model_loss)
    def test(self,X,y):
        
        temp=self.forward(X)
        holder=[]
        for i in temp:
            i=list(i)
            holder.append(i.index(max(i)))
        tt = sum(1 for x, z in zip(holder, y) if x == z)
        print((tt/len(holder))*100)

# Preparing Dataset

In [329]:
data=datasets.load_iris()
x,y,names=data['data'],data['target'],data["target_names"]

# Splitting the Dataset into Training and Testing

In [330]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,shuffle=True)

# Training the Model 

In [331]:
a=NeuralNetwork()
a.train(x_train,y_train,2,[5,3])

446.7748298093375
0.30507064904800707
0.18133778942878642
0.1735769170961916
0.10343510778420395
0.09240511238898641
0.10572184955385566
0.07738839544607856
0.16674687460594306
0.05837133431515262
0.14216171728630478
0.048739384764211025
0.08614257973317407
0.07536117514520413
0.06524567936800807
0.05890605210416133
0.049696380661291784
0.04112907518251063
0.033336156839508746
0.030292337024555138
0.030334796125012886
0.03196358704887459
0.16363152119947344
0.029528238439780807
0.029339115109271674
0.2193427652186415
0.10344473803632681
0.22873616604487523
0.029059768600511404
0.029218820097272058
0.047618902115736716
0.03917005106348803
0.032215172984651375
0.10682258084485774
0.04099818481008022
0.03078838795678962
0.11850175522896222
0.03209933845902668
0.029649232624890942
0.02941902223262005
0.11916298139433579
0.029146216304964057
0.05077367198617999
0.02935541594486679
0.02932870894660411
0.029308855682938173
0.03156744975793835
0.02910708237325821
0.05380571006183388
0.02894672

# Testing the Model

In [332]:
a.test(x_test,y_test)

100.0
